In [86]:
import sys
sys.path.append('../')
import pandas as pd


from utils.GetData import Data2
import numpy as np
from sklearn.model_selection import train_test_split

In [68]:
PATH = '../in/train.csv'
df = pd.read_csv(PATH)

In [69]:
data = Data2()
dfx = data.get_data2(df.copy())

In [70]:
feature = dfx.drop('SalePrice', axis=1)
y = dfx['SalePrice']

In [71]:
pred_example = pd.read_csv('../in/sample_submission.csv')
test = pd.read_csv('../in/test.csv')
test_df = data.get_data2(test.copy(), fit=False)
test_df.shape

(1459, 207)

In [72]:
feature['GarageYrBlt'] = feature['GarageYrBlt'].astype('float')
test_df['GarageYrBlt'] = test_df['GarageYrBlt'].astype('float')

In [73]:
high_skew = ['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', '3SsnPorch',
       'KitchenAbvGr', 'BsmtFinSF2', 'EnclosedPorch', 'ScreenPorch',
       'BsmtHalfBath', 'OpenPorchSF', 'WoodDeckSF', '1stFlrSF', 'BsmtFinSF1',
       'MSSubClass', 'GrLivArea', 'TotalBsmtSF', 'BsmtUnfSF', '2ndFlrSF',
       'TotRmsAbvGrd', 'Fireplaces', 'HalfBath', 'BsmtFullBath', 'OverallCond',
       'YearBuilt']

In [74]:
for ft in high_skew:
    feature[ft] = np.log1p(feature[ft])

feature['TotalSF'] = feature['TotalBsmtSF'] + feature['1stFlrSF'] + feature['2ndFlrSF']
#feature.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)

In [75]:
for ft in high_skew:
    test_df[ft] = np.log1p(test_df[ft])

test_df['TotalSF'] = test_df['TotalBsmtSF'] + test_df['1stFlrSF'] + test_df['2ndFlrSF']
#test_df.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)

In [76]:
y = np.log1p(y)

In [97]:
X_train, X_valid, y_train, y_valid = train_test_split(feature, y, test_size=0.15, random_state=7)

In [98]:
import xgboost as XGB

the_model = XGB.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.01, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=3000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, random_state =7, nthread = -1,
                             eval_set=[(X_valid, y_valid)], verbose=False,
                             early_stopping_rounds=5)
the_model.fit(X_train, y_train)

[00:42:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "early_stopping_rounds", "eval_set", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4603,
             early_stopping_rounds=5, enable_categorical=False,
             eval_set=[(      MSSubClass  LotFrontage   LotArea  LotShape  Utilities  LandSlope  \
606     3.044522         82.0  9.430680       1.0        3.0        1.0   
1210    4.110874         70.0  9.325364       3.0        3.0        2.0   
492     4.110874        105.0  9.653679       2.0        3.0        2.0   
410     3.044522         68.0  9.166597       3.0        3.0...
             gamma=0.0468, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1.7817, missing=nan,
             monotone_constraints='()', n_estimators=3000, n_jobs=4, nthread=-1,
             num_parallel_tree=1, predictor='auto', random_state=7,
             reg_alpha=0.464, reg_lambda=0.8571, scale_pos_weight=1,
 

In [99]:
y_predict = np.floor(np.expm1(the_model.predict(test_df)))


In [100]:
sub = pd.DataFrame()
sub['Id'] = pred_example.Id
sub['SalePrice'] = y_predict
sub.to_csv('../out/mysubmission6.csv',index=False)
sub = sub.set_index('Id')

In [101]:
sub.head()

,SalePrice
Id,
1461,124009.0
1462,161149.0
1463,182254.0
1464,189969.0
1465,192938.0


In [82]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor(hidden_layer_sizes=(338, 338, 50), 
                  activation='relu', solver='adam', max_iter = 100, random_state = 56, verbose = True)
nn.fit(feature, y)
nn_predictions = nn.predict(test_df)

c:\Users\daniel\Desktop\practica\Advanced_Regression_Techniques\renv\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Iteration 1, loss = 1871.08774481
Iteration 2, loss = 74.41649992
Iteration 3, loss = 21.85355738
Iteration 4, loss = 9.64653208
Iteration 5, loss = 4.99194223
Iteration 6, loss = 2.09621942
Iteration 7, loss = 1.27952006
Iteration 8, loss = 0.63993198
Iteration 9, loss = 0.46749957
Iteration 10, loss = 0.31470204
Iteration 11, loss = 0.28703911
Iteration 12, loss = 0.27679930
Iteration 13, loss = 0.17075251
Iteration 14, loss = 0.15098892
Iteration 15, loss = 0.13990893
Iteration 16, loss = 0.12477852
Iteration 17, loss = 0.17255184
Iteration 18, loss = 0.16656872
Iteration 19, loss = 0.13395276
Iteration 20, loss = 0.09065124
Iteration 21, loss = 0.08492171
Iteration 22, loss = 0.13575938
Iteration 23, loss = 0.79718374
Iteration 24, loss = 0.41206351
Iteration 25, loss = 0.18698757
Iteration 26, loss = 0.12734291
Iteration 27, loss = 0.11445288
Iteration 28, loss = 0.09026174
Iteration 29, loss = 0.07492513
Iteration 30, loss = 0.07978851
Iteration 31, loss = 0.12238269
Iteration 32

c:\Users\daniel\Desktop\practica\Advanced_Regression_Techniques\renv\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [83]:
y_predict2 = np.expm1(nn_predictions)

In [85]:
sub = pd.DataFrame()
sub['Id'] = pred_example.Id
sub['SalePrice'] = y_predict2
sub.to_csv('../out/mysubmission5.csv',index=False)
sub = sub.set_index('Id')
sub.head()

,SalePrice
Id,
1461,124275.415970
1462,114164.333259
1463,212044.704943
1464,209025.222462
1465,194878.253935
